In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = './cifar-10-batches-py'
print("1. Structure of CIFAR_DIR {0}".format(os.listdir(CIFAR_DIR)))

1. Structure of CIFAR_DIR ['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def loadData(filename):
    '''
    Read data from filename
    '''
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding = 'bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    '''
    CifarData is a wrapper that process CIFAR-10 Data
    '''
    def __init__(self, filenames, need_shuffle):
        '''
        The constructor takes in a list of files and read all data and labels into self._data member
        @param filenames - list of files that contains all data
        @param need_shuffle - a boolean flag indicating whether order of data needs to be shuffled. 
                              `True` of training set and `False` for testing set
        '''
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = loadData(filename)
            all_data.append(data)
            all_labels.append(labels)
        
        self._data = np.vstack(all_data) / 127.5 - 1
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
            # When we process the data by Mini Batch, this is a cursor indicating where
            # we are in the dataset
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        '''
        Get a random permutation of all indices between 0 and _num_examples, and apply 
        this random permutation of indices to `self._data` and `self._labels`.
        '''
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        '''
        Generate a batch of data for processing. For training set, if end index is larger 
        than `self._num_examples`, reshuffle the data and start over from index 0. For 
        testing set, if end index is out of bound, raise an exception.
        @param batch_size - Number of samples for the batch of data and labels to return
        '''
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("Do not have enough testing samples for next batch")
                
        if end_indicator > self._num_examples:
            raise Exception("Batch size is larger than _num_examples")
            
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    def reset(self):
        '''
        `_shuffle_data` and reset `_indicator` to 0
        '''
        self._shuffle_data()
        self._indicator = 0
    
    def getDataShape(self):
        return self._data.shape
    
    def getLabelShape(self):
        return self._labels.shape
    
print("2. Basic test for `loadData` and `CifarData` class.")
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range (1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
print("Train data shape = {0}, labels shape = {1}".
      format(train_data.getDataShape(), train_data.getLabelShape()))

test_data = CifarData(test_filenames, False)
print("Test data shape = {0}, labels shape = {1}".
      format(test_data.getDataShape(), test_data.getLabelShape()))

batch_data, batch_labels = train_data.next_batch(10)
print("Batch Data = {0}, Batch Labels = {1}".format(batch_data, batch_labels))

2. Basic test for `loadData` and `CifarData` class.
Train data shape = (50000, 3072), labels shape = (50000,)
Test data shape = (10000, 3072), labels shape = (10000,)
Batch Data = [[ 0.96862745  0.95294118  0.96862745 ...  0.92156863  0.92156863
   0.92156863]
 [ 0.78039216  0.79607843  0.76470588 ... -0.58431373 -0.56862745
  -0.52156863]
 [-0.67058824 -0.63921569 -0.65490196 ... -0.61568627 -0.49803922
  -0.52941176]
 ...
 [ 0.55294118  0.52941176  0.45098039 ... -0.41176471 -0.41960784
  -0.39607843]
 [-0.51372549 -0.5372549  -0.56862745 ...  0.96078431  0.94509804
   0.95294118]
 [ 0.05098039 -0.05882353 -0.0745098  ...  0.27058824  0.34901961
   0.37254902]], Batch Labels = [5 7 4 6 9 0 2 9 6 4]


In [3]:
# Define x and y
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

# Define weights w and intercept b
# w shape is (3072, 10) and it is initialized as a randome value distributed normally
# b shape is (10,) and all of them are initialized as 0.0
w = tf.get_variable('w', [x.get_shape()[-1], 10], initializer = tf.random_normal_initializer(0, 1))
b = tf.get_variable('b', [10], initializer = tf.constant_initializer(0.0))

# y_pred, predict and y_actual are both [None, 10] shape
z = tf.matmul(x, w) + b
y_prob = tf.nn.softmax(z)
y_predict = tf.argmax(z, 1)
y_actual_oneHot = tf.one_hot(y, 10, dtype = tf.float32)

# Calculate loss and accuracy
loss = tf.reduce_mean(tf.square(y_actual_oneHot - y_prob))
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_predict, y), tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
print("3. Apply TensorFlow model to CIFAR-10 Data.")
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
train_data.reset()

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op],
                                        feed_dict = {x: batch_data, y: batch_labels})
        # Every 500 steps print loss and accuracy
        if (i + 1) % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, accuracy: %4.5f' \
                 % (i + 1, loss_val, acc_val))
        
        # Every 5,000 steps run model against test model for accuracy
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_accuracies = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], 
                                        feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_accuracies.append(test_acc_val)
            test_accuracy = np.mean(all_test_accuracies)
            print('[Test ] step: %d, accuracy: %4.5f' % (i + 1, test_accuracy))

3. Apply TensorFlow model to CIFAR-10 Data.
[Train] step: 500, loss: 0.16699, accuracy: 0.15000
[Train] step: 1000, loss: 0.11684, accuracy: 0.35000
[Train] step: 1500, loss: 0.13775, accuracy: 0.30000
[Train] step: 2000, loss: 0.15708, accuracy: 0.15000
[Train] step: 2500, loss: 0.11717, accuracy: 0.40000
[Train] step: 3000, loss: 0.12681, accuracy: 0.30000
[Train] step: 3500, loss: 0.16540, accuracy: 0.15000
[Train] step: 4000, loss: 0.14813, accuracy: 0.25000
[Train] step: 4500, loss: 0.14345, accuracy: 0.25000
[Train] step: 5000, loss: 0.13610, accuracy: 0.30000
[Test ] step: 5000, accuracy: 0.29800
[Train] step: 5500, loss: 0.12145, accuracy: 0.35000
[Train] step: 6000, loss: 0.12844, accuracy: 0.35000
[Train] step: 6500, loss: 0.15402, accuracy: 0.15000
[Train] step: 7000, loss: 0.08005, accuracy: 0.60000
[Train] step: 7500, loss: 0.16308, accuracy: 0.15000
[Train] step: 8000, loss: 0.12711, accuracy: 0.35000
[Train] step: 8500, loss: 0.13864, accuracy: 0.30000
[Train] step: 9000

[Train] step: 71500, loss: 0.12469, accuracy: 0.35000
[Train] step: 72000, loss: 0.11765, accuracy: 0.40000
[Train] step: 72500, loss: 0.13404, accuracy: 0.30000
[Train] step: 73000, loss: 0.10596, accuracy: 0.45000
[Train] step: 73500, loss: 0.10815, accuracy: 0.45000
[Train] step: 74000, loss: 0.07848, accuracy: 0.60000
[Train] step: 74500, loss: 0.12772, accuracy: 0.30000
[Train] step: 75000, loss: 0.12808, accuracy: 0.35000
[Test ] step: 75000, accuracy: 0.38200
[Train] step: 75500, loss: 0.14697, accuracy: 0.25000
[Train] step: 76000, loss: 0.11147, accuracy: 0.45000
[Train] step: 76500, loss: 0.12560, accuracy: 0.35000
[Train] step: 77000, loss: 0.11926, accuracy: 0.40000
[Train] step: 77500, loss: 0.08922, accuracy: 0.55000
[Train] step: 78000, loss: 0.14070, accuracy: 0.30000
[Train] step: 78500, loss: 0.12805, accuracy: 0.35000
[Train] step: 79000, loss: 0.11603, accuracy: 0.40000
[Train] step: 79500, loss: 0.13902, accuracy: 0.30000
[Train] step: 80000, loss: 0.12540, accurac

In [5]:
print("4. After changing loss function to be cross entropy instead of mean square.")
loss_crossEntropy = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = z)

init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
train_data.reset()

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss_crossEntropy, accuracy, train_op],
                                        feed_dict = {x: batch_data, y: batch_labels})
        # Every 500 steps print loss and accuracy
        if (i + 1) % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, accuracy: %4.5f' \
                 % (i + 1, loss_val, acc_val))
        
        # Every 5,000 steps run model against test model for accuracy
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_accuracies = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], 
                                        feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_accuracies.append(test_acc_val)
            test_accuracy = np.mean(all_test_accuracies)
            print('[Test ] step: %d, accuracy: %4.5f' % (i + 1, test_accuracy))

3. After changing loss function to be cross entropy instead of mean square.
[Train] step: 500, loss: 37.66614, accuracy: 0.15000
[Train] step: 1000, loss: 22.43869, accuracy: 0.25000
[Train] step: 1500, loss: 30.23055, accuracy: 0.10000
[Train] step: 2000, loss: 48.73115, accuracy: 0.20000
[Train] step: 2500, loss: 42.26724, accuracy: 0.30000
[Train] step: 3000, loss: 40.12151, accuracy: 0.25000
[Train] step: 3500, loss: 21.87340, accuracy: 0.60000
[Train] step: 4000, loss: 23.38885, accuracy: 0.20000
[Train] step: 4500, loss: 35.25844, accuracy: 0.40000
[Train] step: 5000, loss: 37.09795, accuracy: 0.10000
[Test ] step: 5000, accuracy: 0.30400
[Train] step: 5500, loss: 29.41338, accuracy: 0.40000
[Train] step: 6000, loss: 30.98164, accuracy: 0.30000
[Train] step: 6500, loss: 57.37870, accuracy: 0.05000
[Train] step: 7000, loss: 14.33826, accuracy: 0.50000
[Train] step: 7500, loss: 26.90653, accuracy: 0.35000
[Train] step: 8000, loss: 33.27259, accuracy: 0.50000
[Train] step: 8500, los

[Train] step: 70000, loss: 20.04059, accuracy: 0.55000
[Test ] step: 70000, accuracy: 0.39900
[Train] step: 70500, loss: 23.18429, accuracy: 0.55000
[Train] step: 71000, loss: 29.57625, accuracy: 0.30000
[Train] step: 71500, loss: 31.32345, accuracy: 0.30000
[Train] step: 72000, loss: 45.68279, accuracy: 0.25000
[Train] step: 72500, loss: 34.26932, accuracy: 0.30000
[Train] step: 73000, loss: 32.60050, accuracy: 0.35000
[Train] step: 73500, loss: 33.73286, accuracy: 0.35000
[Train] step: 74000, loss: 23.93733, accuracy: 0.50000
[Train] step: 74500, loss: 37.70154, accuracy: 0.50000
[Train] step: 75000, loss: 34.61103, accuracy: 0.50000
[Test ] step: 75000, accuracy: 0.39500
[Train] step: 75500, loss: 40.43531, accuracy: 0.50000
[Train] step: 76000, loss: 20.42539, accuracy: 0.50000
[Train] step: 76500, loss: 52.07831, accuracy: 0.20000
[Train] step: 77000, loss: 34.71130, accuracy: 0.35000
[Train] step: 77500, loss: 17.23452, accuracy: 0.45000
[Train] step: 78000, loss: 54.52991, accur